In [7]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

In [25]:
train = pd.read_csv('train.csv',index_col='id')
train

,type,fiberID,psfMag_u,psfMag_g,psfMag_r,psfMag_i,psfMag_z,fiberMag_u,fiberMag_g,fiberMag_r,...,petroMag_u,petroMag_g,petroMag_r,petroMag_i,petroMag_z,modelMag_u,modelMag_g,modelMag_r,modelMag_i,modelMag_z
id,,,,,,,,,,,,,,,,,,,,,
0,QSO,601,23.198224,21.431953,21.314148,21.176553,21.171444,22.581309,21.644453,21.657571,...,22.504317,21.431636,21.478312,21.145409,20.422446,22.749241,21.465534,21.364187,21.020605,21.147340
1,QSO,788,21.431355,20.708104,20.678850,20.703420,20.473229,21.868797,21.029773,20.967054,...,21.360701,20.778968,20.889705,20.639812,20.646660,21.492955,20.758527,20.753925,20.693389,20.512314
2,QSO,427,17.851451,16.727898,16.679677,16.694640,16.641788,18.171890,17.033098,16.999682,...,17.867253,16.738784,16.688874,16.744210,16.808006,17.818063,16.697434,16.641249,16.660177,16.688928
3,QSO,864,20.789900,20.040371,19.926909,19.843840,19.463270,21.039030,20.317165,20.217898,...,20.433907,19.993727,19.985531,19.750917,19.455117,20.770711,20.001699,19.889798,19.758113,19.552855
4,STAR_RED_DWARF,612,26.454969,23.058767,21.471406,19.504961,18.389096,25.700632,23.629122,21.742750,...,25.859229,22.426929,21.673551,19.610012,18.376141,24.877052,23.147993,21.475342,19.487330,18.375655
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199986,QSO,568,20.630780,20.437920,20.050756,20.020974,19.959646,20.948519,20.726709,20.311562,...,20.836400,20.536338,20.126561,20.102815,19.796722,20.783295,20.326879,20.014046,20.061940,19.862513
199987,GALAXY,362,25.283695,22.807243,21.069792,20.187531,19.595598,26.677671,22.818723,20.786120,...,28.895442,22.250489,20.091693,18.985791,18.803354,26.921903,21.952440,19.776321,18.990039,18.528306
199988,STAR_RED_DWARF,146,23.609826,21.902709,20.330747,18.457535,17.326771,24.320816,22.287946,20.711737,...,25.982378,22.132993,20.329624,18.507337,17.418521,24.503460,21.897599,20.420524,18.443126,17.331724


Catboost는 parameter를 설정하지 않기 때문에 이를 돌린다.
Scaler중에서 뭐가 적합한지 보기 위해 4가지 Scaler를 쓰는데 동일한 조건으로 catboost를 돌려서 log loss를 비교하고 가장 작은 값을 도출하는 Scaler를 선택한다.
FiberID는 제외하고 Scaling한 후에 다시 fiberID를 합친다.

In [9]:
# RobustScaler
from sklearn.preprocessing import RobustScaler
train_for_scaling = train.iloc[:,2:] #type과 fiberID 제외하고 데이터 생성
robustScaler = RobustScaler()
print(robustScaler.fit(train_for_scaling))
train_robustscaled = robustScaler.transform(train_for_scaling) #robustscaling 적용
train_robustscaled = pd.DataFrame(train_robustscaled,columns=train_for_scaling.columns)
train_robustscaled = pd.concat([train[['type', 'fiberID']], train_robustscaled], axis=1) 
train_robustscaled #robustscaling과 type,fiberID 합치기

#Train, test 쪼개기
X_robustscaled = train_robustscaled.drop(['type'], axis = 1)
y_robustscaled = train_robustscaled['type']
X_train_robustscaled, X_test_robustscaled, y_train_robustscaled, y_test_robustscaled = train_test_split(X_robustscaled, y_robustscaled, test_size= 0.2, random_state=42)

RobustScaler(copy=True, quantile_range=(25.0, 75.0), with_centering=True,
       with_scaling=True)


,type,fiberID,psfMag_u,psfMag_g,psfMag_r,psfMag_i,psfMag_z,fiberMag_u,fiberMag_g,fiberMag_r,...,petroMag_u,petroMag_g,petroMag_r,petroMag_i,petroMag_z,modelMag_u,modelMag_g,modelMag_r,modelMag_i,modelMag_z
0,QSO,601,0.927836,0.623783,0.753702,0.916931,1.041252,0.639184,0.634486,0.805768,...,0.838327,0.638332,0.770905,0.825931,0.748549,0.859066,0.664478,0.744135,0.795289,0.978342
1,QSO,788,0.223298,0.328228,0.496221,0.713509,0.757221,0.341950,0.386929,0.531161,...,0.389795,0.412532,0.573234,0.655629,0.823214,0.398328,0.419522,0.539673,0.685895,0.774402
2,QSO,427,-1.204186,-1.296932,-1.124607,-1.010056,-0.801392,-1.200267,-1.222701,-1.046600,...,-0.980354,-0.985231,-0.837531,-0.656544,-0.455081,-0.949423,-0.987518,-0.838240,-0.662475,-0.453490
3,QSO,864,-0.032482,0.055586,0.191466,0.343934,0.346375,-0.004199,0.099931,0.233233,...,0.026301,0.140866,0.269585,0.356218,0.426423,0.133448,0.157305,0.250155,0.373217,0.466269
4,STAR_RED_DWARF,612,2.226460,1.288028,0.817437,0.198234,-0.090595,1.940455,1.433796,0.839643,...,2.154141,0.982669,0.836473,0.308756,0.067117,1.639432,1.247397,0.781377,0.282689,0.088207
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199986,QSO,568,-0.095930,0.217909,0.241661,0.420093,0.548298,-0.041957,0.264872,0.270482,...,0.184161,0.328591,0.316947,0.474749,0.540180,0.138063,0.269970,0.291783,0.474791,0.565716
199987,GALAXY,362,1.759416,1.185329,0.654667,0.491704,0.400205,2.348040,1.107414,0.459206,...,3.344960,0.921627,0.305237,0.098497,0.209382,2.389373,0.833175,0.212136,0.116437,0.137232
199988,STAR_RED_DWARF,146,1.091962,0.815997,0.355138,-0.252104,-0.522744,1.364844,0.893648,0.429626,...,2.202441,0.880978,0.385142,-0.062662,-0.251776,1.502419,0.814175,0.427970,-0.066405,-0.247054
199989,GALAXY,85,0.119882,0.268249,0.233959,0.322353,0.377780,0.273255,0.161383,0.095711,...,-0.582843,-0.526213,-0.564692,-0.509540,-0.332802,-0.475946,-0.534496,-0.570515,-0.484095,-0.363270


In [10]:
# StandardScaler
from sklearn.preprocessing import StandardScaler
standardScaler = StandardScaler()
print(standardScaler.fit(train_for_scaling))
train_standardscaled = standardScaler.transform(train_for_scaling)
train_standardscaled = pd.DataFrame(train_standardscaled,columns=train_for_scaling.columns)
train_standardscaled = pd.concat([train[['type', 'fiberID']], train_standardscaled], axis=1) 
train_standardscaled #standardscaling과 type,fiberID 합치기

X_standardscaled = train_standardscaled.drop(['type'], axis = 1)
y_standardscaled = train_standardscaled['type']
X_train_standardscaled, X_test_standardscaled, y_train_standardscaled, y_test_standardscaled = train_test_split(X_standardscaled, y_standardscaled, test_size= 0.2, random_state=42)

In [2]:
# MinMaxScaler
from sklearn.preprocessing import MinMaxScaler
minmaxscaler = MinMaxScaler()
print(minmaxscaler.fit(train_for_scaling))
train_minmaxscaled = minmaxscaler.transform(train_for_scaling)
train_minmaxscaled = pd.DataFrame(train_minmaxscaled,columns=train_for_scaling.columns)
train_minmaxscaled = pd.concat([train[['type', 'fiberID']], train_minmaxscaled], axis=1) 
train_minmaxscaled #minmaxscaling과 type,fiberID 합치기

X_minmaxscaled = train_minmaxscaled.drop(['type'], axis = 1)
y_minmaxscaled = train_minmaxscaled['type']
X_train_minmaxscaled, X_test_minmaxscaled, y_train_minmaxscaled, y_test_minmaxscaled = train_test_split(X_minmaxscaled, y_minmaxscaled, test_size= 0.2, random_state=42)

In [35]:
# MaxAbsScaler
from sklearn.preprocessing import MaxAbsScaler
maxabsscaler = MaxAbsScaler()
print(maxabsscaler.fit(train_for_scaling))
train_maxabsscaled = maxabsscaler.transform(train_for_scaling)
train_maxabsscaled = pd.DataFrame(train_maxabsscaled,columns=train_for_scaling.columns)
train_maxabsscaled = pd.concat([train[['type', 'fiberID']], train_maxabsscaled], axis=1) 
train_maxabsscaled #maxabsscaling과 type,fiberID 합치기

X_maxabsscaled = train_maxabsscaled.drop(['type'], axis = 1)
y_maxabsscaled = train_maxabsscaled['type']
X_train_maxabsscaled, X_test_maxabsscaled, y_train_maxabsscaled, y_test_maxabsscaled = train_test_split(X_maxabsscaled, y_maxabsscaled, test_size= 0.2, random_state=42)

0.8579964499112478

In [1]:
#Catboost & Log Loss
import catboost as cb
from sklearn.metrics import log_loss

cb_model = cb.CatBoostClassifier()

cb_model.fit(X_train_robustscaled,y_train_robustscaled)
y_pred_prob_cb_robustscaled = cb_model.predict_proba(X_test_robustscaled)

cb_model.fit(X_train_standardscaled,y_train_standardscaled)
y_pred_prob_cb_standardscaled = cb_model.predict_proba(X_test_standardscaled)

cb_model.fit(X_train_minmaxscaled,y_train_minmaxscaled)
y_pred_prob_cb_minmaxscaled = cb_model.predict_proba(X_test_minmaxscaled)

cb_model.fit(X_train_maxabsscaled,y_train_maxabsscaled)
y_pred_prob_cb_maxabsscaled = cb_model.predict_proba(X_test_maxabsscaled)

In [60]:
print("Robust: " , log_loss(y_true=y_test_robustscaled, y_pred=y_pred_prob_cb_robustscaled))
print("Standard: " , log_loss(y_true=y_test_standardscaled, y_pred=y_pred_prob_cb_standardscaled))
print("MinMax: " , log_loss(y_true=y_test_minmaxscaled, y_pred=y_pred_prob_cb_minmaxscaled))
print("MaxAbs: " , log_loss(y_true=y_test_maxabsscaled, y_pred=y_pred_prob_cb_maxabsscaled))

Robust:  0.42774662154063664
Standard:  0.4278871022373594
MinMax:  0.4305520118888065
MaxAbs:  0.4277657503242839


In [ ]:
#SMOTE
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
X_smote, y_smote = SMOTE(random_state=4).fit_sample(X, y)
#SMOTE를 진행하면 성능이 더 안 좋아진다

In [ ]:
#Sampling
orb_type = [
    "GALAXY",
    "QSO",
    "REDDEN_STD",
    "ROSAT_D",
    "SERENDIPITY_BLUE",
    "SERENDIPITY_DISTANT",
    "SERENDIPITY_FIRST",
    "SERENDIPITY_MANUAL",
    "SERENDIPITY_RED",
    "SKY",
    "SPECTROPHOTO_STD",
    "STAR_BHB",
    "STAR_BROWN_DWARF",
    "STAR_CARBON",
    "STAR_CATY_VAR",
    "STAR_PN",
    "STAR_RED_DWARF",
    "STAR_SUB_DWARF",
    "STAR_WHITE_DWARF"
]
def sampling(df, orb_type): #df : 데이터프레임, orb_type : 천체타입 리스트
    copy_train = df.copy()
    for t in orb_type:
        print("현재 type : ", t)
        insert_row = int(input("랜덤샘플링으로 추가할 개수를 입력하세요 : "))
        print("\n")
        temp = df[df["type"]==t].sample(n = insert_row, replace=True)  #t는 tpye의 종류, replace = True 옵션을 줘야 개수가 더 많아도 샘플링 가능
        copy_train = pd.concat([copy_train, temp], axis = 0) #row로 붙여넣기
    return copy_train

In [6]:
#Sampling 구현
data = sampling(data, ["STAR_PN", "SERENDIPITY_MANUAL", "SKY", "STAR_BROWN_DWARF"])